In [1]:
import tensorflow as tf
import numpy as np
import os
import time

# seqeunce.take(5) 문법 변경 문제 해결
tf.enable_eager_execution()

In [3]:
from google.colab import files
uploaded = files.upload()

Saving sherlock_homes.txt to sherlock_homes (1).txt


In [4]:
with open('sherlock_homes.txt', 'r') as file:
    text = file.read().lower()

vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [5]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
p
r
o
j


In [6]:
# batch 메서드는 이 개별 문자들을 원하는 크기의 시퀀스로 쉽게 변환할 수 있습니다.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

# train / test split
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
# (TF 데이터는 무한한 시퀀스와 함께 작동이 가능하도록 설계되었으며,
# 따라서 전체 시퀀스를 메모리에 섞지 않습니다. 대신에,
# 요소를 섞는 버퍼를 유지합니다).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

"\ufeffproject gutenberg's the adventures of sherlock holmes, by arthur conan doyle\n\nthis ebook is for the "
'use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give '
'it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or onl'
'ine at www.gutenberg.net\n\n\ntitle: the adventures of sherlock holmes\n\nauthor: arthur conan doyle\n\npost'
'ing date: april 18, 2011 [ebook #1661]\nfirst posted: november 29, 2002\n\nlanguage: english\n\n\n*** start'


<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

epoch = 30


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 
                              embedding_dim,
                              batch_input_shape = [batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences = True,
                        stateful = True,
                        recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
vocab_size = len(vocab),
embedding_dim=embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16128     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 63)            64575     
Total params: 5,327,679
Trainable params: 5,327,679
Non-trainable params: 0
_________________________________________________________________


In [12]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
#   print(example_batch_predictions)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(64, 100, 63) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


In [13]:
# 손실함수 정의
# 출력값이 로짓의 형태이기 때문에 from_logit = True

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 63)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           4.1420484


In [0]:
# tf.keras.Model.compile 메서드를 사용하여 훈련 절차를 설정합니다. 
# 기본 매개변수의 tf.keras.optimizers.Adam과 손실 함수를 사용합니다.

model.compile(optimizer='adam', loss=loss)

In [0]:
#체크포인트 구성
# tf.keras.callbacks.ModelCheckpoint를 사용하여 훈련 중 체크포인트(checkpoint)가 저장되도록 합니다.

# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [17]:
history = model.fit(dataset, epochs=epoch, callbacks=[checkpoint_callback])

Epoch 1/30
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
90/90 [==============================] - 20s 225ms/step - loss: 2.9287
Epoch 2/30
90/90 [==============================] - 18s 205ms/step - loss: 2.3743
Epoch 3/30
90/90 [==============================] - 18s 205ms/step - loss: 2.1546
Epoch 4/30
90/90 [==============================] - 18s 203ms/step - loss: 1.9670
Epoch 5/30
90/90 [==============================] - 18s 203ms/step - loss: 1.8251
Epoch 6/30
90/90 [==============================] - 18s 203ms/step - loss: 1.7177
Epoch 7/30
90/90 [==============================] - 18s 204ms/step - loss: 1.6311
Epoch 8/30
90/90 [==============================] - 18s 204ms/step - loss: 1.5591
Epoch 9/30
90/90 [==============================] - 18s 204ms/step - loss: 1.4990
Epoch 10/30
90/90 [==============================] - 18s 205ms/step - loss: 1.4495
Epoch 11/30
90/90 [==============================] - 18s 202ms/step - loss: 1.4059
Ep

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
  num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
  # 온도가 높으면 더 의외의 텍스트가 됩니다.
  # 최적의 세팅을 찾기 위한 실험
  temperature = 1.0

  # 여기에서 배치 크기 == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 배치 차원 제거
      predictions = tf.squeeze(predictions, 0)

      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 예측된 단어를 다음 입력으로 모델에 전달
      # 이전 은닉 상태와 함께
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [21]:
print(generate_text(model, start_string=u"homes"))

homes before which shrunge
over with the other. now i will sold you the comntionalock, and how we drove it all darrats in uncincitance with these
steps. nd not mame. i had excelled them.

"missing had to come at and easy, coming downstairs, so that the
pool. i think that i shall excy matterly. the oneycripoldight,
which is our times. if he wished to refer of their
florger. i followed them up, there is nothing by the part close
"where she had fallen alive indeed, which he was a monogram of his act.
john, however. i wondempell, returning we had run him as hard as i caught the band, which
broken and aguatione. he cut onf for
visit glasses which might be
the hands had been made a very strange that of compliance will
front of the previnger, by some little
help of his chocked with a nod fellow might collar
into my disappearance, his trifled groubles, was stathed than
usually my hand, came breckive
incledgemen, boots."

he looked demred the village lately startled brightly
binds in a lantures